# sentient-sentiment

This is my research on developing a bidrectional LSTM model trained specifically for financial news.

Datasets from [here](http://jmcauley.ucsd.edu/data/amazon/).

*Author: Calvin Kinateder*

## Set up drive support

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import Modules

In [ ]:
# Importing required libraries
!pip install jsonlines 
import nltk
from keras import backend as K
from scipy.interpolate import interp1d
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import re
import jsonlines

## Set params

In [ ]:
NUM_WORDS = 8192 # number of unique words #(10000)
LSTM_UNITS = 256 # lstm cells #(300)
EMBED_OUT_DIM = 512
NUM_CLASSES = 2 # is overwritten based on number of unique vals in data
NUM_EPOCHS = 8 # keep lower to avoid overfitting
TEXT_COL = "text"
SCORE_COL = "sentiment"
BATCH_SIZE = 512 #(400)
TEST_SIZE = .2 # 
USING_TPU = True
MAX_LENGTH = 0 # unknown at instantiation

# get stop words
stop_words = stopwords.words('english')

## Set up TPU
Training happens much faster here.

In [ ]:
if USING_TPU:
  assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

  if 'COLAB_TPU_ADDR' in os.environ:
    TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
  else:
    TF_MASTER=''

  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)

## Create persistent objects
Tokenizer is used for both training and inference and thus must be kept the same.

In [ ]:
tokenizer = Tokenizer(num_words=NUM_WORDS, split=' ') 
labelEncoder=LabelEncoder()

In [ ]:
imdb = pd.read_csv("/content/drive/MyDrive/datasets/imdb.csv")[["review","sentiment"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})

## Load datasets

In [ ]:
# load jsonlines
def read_jsonl(path:str)->pd.DataFrame:
    out = []
    with jsonlines.open(path) as f:
        for line in f.iter():
            out.append(line)
    return pd.DataFrame(out)

def filter(df: pd.DataFrame, col: str):
  df[col] = np.select(
    [df[col] > 3, df[col] == 3, df[col] < 3], ["positive", "neutral", "negative"]
  )

In [ ]:
# from http://jmcauley.ucsd.edu/data/amazon/ 
amazon_set_1 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Musical_Instruments_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
filter(amazon_set_1, SCORE_COL)

amazon_set_2 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Office_Products_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
filter(amazon_set_2, SCORE_COL)

amazon_set_3 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Tools_and_Home_Improvement_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
filter(amazon_set_3, SCORE_COL)

amazon_set_4 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Toys_and_Games_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
filter(amazon_set_4, SCORE_COL)

amazon_set_5 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Home_and_Kitchen_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
filter(amazon_set_5, SCORE_COL)

#amazon_set_6 = read_jsonl("/content/drive/MyDrive/datasets/reviews_Electronics_5.json")[["summary", "overall"]].rename(columns={"summary": TEXT_COL, "overall":SCORE_COL})
#filter(amazon_set_6, SCORE_COL)

imdb = pd.read_csv("/content/drive/MyDrive/datasets/imdb.csv")[["review","sentiment"]].rename(columns={"review": TEXT_COL, "sentiment":SCORE_COL})

#Loading the finance_set_1set
finance_set_1 = pd.read_csv("/content/drive/MyDrive/datasets/small/all-data.csv", encoding="ISO-8859-1")
finance_set_1.columns = [SCORE_COL, TEXT_COL]

## Create combined dataset

In [ ]:
data = pd.concat([finance_set_1, amazon_set_1, amazon_set_2, amazon_set_3, amazon_set_4, amazon_set_5])
data = data[data[SCORE_COL] != "neutral"] # remove neutral rows cause they do not matter
data = data.sample(frac=1).reset_index(drop=True) # shuffle

## View Distribution

In [ ]:
def show_distribution(data: pd.DataFrame)->None:
  pos_count = data[data[SCORE_COL] == 'positive'].shape[0]
  neg_count = data[data[SCORE_COL] == 'negative'].shape[0]
  fig1, ax1 = plt.subplots()
  ax1.pie([pos_count, neg_count], labels=["positive", "negative"], autopct='%1.1f%%',
          shadow=True, startangle=90)
  ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

  plt.show()
  print(f"Data Positive {pos_count:,} / {data.shape[0]:,} ({(pos_count/data.shape[0])*100:.2f}%)")
  print(f"Data Negative {neg_count:,} / {data.shape[0]:,} ({(neg_count/data.shape[0])*100:.2f}%)")
show_distribution(data)

## Create equal distribution


In [ ]:
def equalize_distribution(data: pd.DataFrame)->pd.DataFrame:
  pos_count = data[data[SCORE_COL] == 'positive'].shape[0]
  neg_count = data[data[SCORE_COL] == 'negative'].shape[0]
  if pos_count > neg_count:
    data[data[SCORE_COL] == 'positive'] = data[data[SCORE_COL] == 'positive'][-neg_count:]
    data = data.dropna()
  elif neg_count > pos_count:
    data[data[SCORE_COL] == 'negative'] = data[data[SCORE_COL] == 'negative'][-pos_count:]
    data = data.dropna()
  return data

#data = equalize_distribution(data)

#show_distribution(data)


## Clean dataset

In [ ]:
def clean(values: pd.Series)->pd.DataFrame: # improve this
    # make all lowercase
    values = values.apply(lambda x: str(x).lower())#' '.join(x.lower() for x in x.split()))
    # Replacing the special characters, digits, numbers
    values = values.apply(lambda x: re.sub('[^A-Za-z]+ ', ' ', x))
    # remove html tags
    values = values.apply(lambda x: re.sub('<.*?>', ' ', x))
    # Lemmatization
    values = values.apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return values
    
data[TEXT_COL] = clean(data[TEXT_COL])
data[SCORE_COL].unique() # should only be "negative", "positive"
data

## Encode and transform sentiment
This takes the unique values and converts into numbers.

In [ ]:
data[SCORE_COL] = labelEncoder.fit_transform(data[SCORE_COL])   

## Tokenize words into X
This, **with the same tokenzier**, is what will be used for processing text for inference.

In [ ]:
# fit first
tokenizer.fit_on_texts(data[TEXT_COL].values)

def find_max_length(sequences: list)->int:
  maxlen = 0
  for seq in sequences:
    if len(seq)> maxlen:
      maxlen = len(seq)
  return maxlen

# define function to create X - this is used later
def create_X(texts: pd.Series):
  sequences = tokenizer.texts_to_sequences(clean(texts).values)
  return pad_sequences(sequences, maxlen=MAX_LENGTH) # set max length here

# create sequences
sequences = tokenizer.texts_to_sequences(data[TEXT_COL].values)

# find max length
MAX_LENGTH = find_max_length(sequences)
print("Max length is", MAX_LENGTH)

X = pad_sequences(sequences, maxlen=MAX_LENGTH) # set max length here

## Create model

### Layers
**Embedding**: Turns positive integers (indexes) into dense vectors of fixed size. *NUM_WORDS* potential word values, *EMBED_OUT_DIM* max length.

**SpatialDropout1D**: Helps reduce noise.

**Bidirectional LSTM**: Most important layer. Is able to "forget" certain things and remember others. Bidirectional LSTMS can understand context better when compared to unidirectional LSTM

**Dense**: Shapes into *NUM_CLASSES*, i.e. 0 for negative and 1 for positive.



In [ ]:
def create_model():  
  model = Sequential()
  model.add(Embedding(NUM_WORDS, EMBED_OUT_DIM, input_length = X.shape[1]))
  model.add(SpatialDropout1D(0.4))
  model.add(SpatialDropout1D(0.3))
  model.add(Bidirectional(LSTM(LSTM_UNITS, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
  model.add(LSTM(int(LSTM_UNITS/2), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(int(LSTM_UNITS/4), dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(NUM_CLASSES, activation='softmax'))
  return model

def compile_model(model: Sequential): #categorical_crossentropy for multiple category flex
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

NUM_CLASSES = data[SCORE_COL].nunique() # reset cols
if USING_TPU:
  with strategy.scope():
    model = create_model()
    compile_model(model)
else:
    model = create_model()
    compile_model(model)
model.summary()

## Split into train and test

In [ ]:
#Splitting the finance_set_1 into training and testing
y=pd.get_dummies(data[SCORE_COL])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=TEST_SIZE, random_state = 42, )

## Fit model
Depending on the system running, this can take an absurd amount of time.

In [ ]:
model.fit(X_train, y_train, epochs = NUM_EPOCHS, batch_size=BATCH_SIZE, verbose = 1)



## Evaluate model
View subsets and evaluate

In [ ]:
acc_and_loss = model.evaluate(X_test,y_test)
loss = acc_and_loss[0]
accuracy = acc_and_loss[1]
print(f"Model loss on test set with {y_test.shape[0]} rows: {loss:.4f}")
print(f"Model accuracy on test set with {y_test.shape[0]} rows: {accuracy*100:0.2f}%")
y_pred = model.predict(X_test)

In [ ]:
subset_x = X_test[-10:]
subset_y = y_test[-10:]

In [ ]:
subset_y_pred = model.predict(subset_x)

In [ ]:
subset_y_pred.round()

In [ ]:
subset_y

In [ ]:
subset_x

In [ ]:
def score(*args)->dict: # predict score for a text input, and return % positive
  interp = interp1d([0,1],[-1,1])
  values = {}
  y = model.predict(create_X(pd.Series([*args])))
  for i in range(len(args)):
    values[args[i]] = round(float(interp(y[i][1])), 4)
  return values


score("Apple's sales dropped today", "the most amazing product", "your mom stinks", "i love your mom")

In [ ]:
model.save("/content/drive/MyDrive/sentient-sentiment-v1")